# Keras YOLO - Autonomous driving & Car detection

YOLO is a popular object detection algoritm because it achieves high accuracy while also being able to run in **real-time**.  
YOLO is an abbreviation of "You Only Look Once". Some algorithms to detect objects in an image use sliding windows to 'scan' the image for objects. YOLO divides an image into a S x S grid of cells and 'only looks once' at all the grid cells, thus without sliding windows. This algorithm "only looks once" at the image in the sense that it requires only one forward propagation pass through the network to make predictions. For each cell the model returns a prediction containing: 
 - a probability(confidence) that an object is in the cell
 - a class probability distribution
 - a fixed number of boundary boxes(x, y, w, h) and it's box probability distribution

You will learn about object detection using the very powerful YOLO model. Many of the ideas in this notebook are described in the two YOLO papers: Redmon et al., 2016 (https://arxiv.org/abs/1506.02640) and Redmon and Farhadi, 2016 (https://arxiv.org/abs/1612.08242). 

**You will learn to**:
- Use object detection on a car detection dataset
- Deal with bounding boxes

##### NOTE: The Yolo model used for prediction is using different parameter sizes/shapes:
 - an input size of 41 x 416, not 608 x 608
 - a raster of 13 x 13 pixels, not 19 x 19
 
Further reads:  
https://pjreddie.com/darknet/yolo/  
https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088     
https://medium.com/@akarshzingade/logo-detection-using-yolov2-8cda5a68740e  
https://arxiv.org/pdf/1506.02640.pdf   
https://arxiv.org/pdf/1612.08242.pdf   
https://github.com/pjreddie/darknet   

In [ ]:
# import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
# import scipy.io

import numpy as np
import pandas as pd
import PIL
from PIL import Image, ImageDraw, ImageFont

from imageio import imread
import cv2

import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model

from mymods.lauthom import *

%matplotlib inline

In [ ]:
from IPython.display import HTML
from moviepy.editor import VideoFileClip
import pygame

### Convolution format/shape

Convolutional kernels will have 2 types of image format/shape:
 - "th" => depth, input_depth, rows, cols
 - "tf" => rows, cols, input_depth, depth

Therefore you can convert from the former to the later via np.transpose(x, (2, 3, 1, 0))

In [ ]:
# keras.backend.set_image_dim_ordering('th')

In [ ]:
class YOLO(object):
    def __init__(self):
        self.model_path = 'model_data/trained_weights.h5' # model path or trained weights path
        self.anchors_path = 'model_data/yolo_anchors.txt'
        #self.classes_path = 'model_data/coco_classes.txt'
        self.classes_path = 'model_data/voc_classes.txt'
        
# To makesure using yolo_body, I modified codes as followed:

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting

        # try:
        #     self.yolo_model = load_model(model_path, compile=False)
        # except:
        #     self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
        #         if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
        #     self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        # else:
        #     assert self.yolo_model.layers[-1].output_shape[-1] == \
        #         num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
        #         'Mismatch between model and given anchor and class sizes'

        self.yolo_model = yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
        self.yolo_model.load_weights("model_data/trained_weights.h5")
        


### Custom modules

In [ ]:
get_path('pycode', 'yolo_utils', 'py', True)
get_path('pycode/YAD2K/models', 'keras_yolo', 'py', True)

In [ ]:
# from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

**Important Note**: As you can see, we import Keras's backend as K. This means that to use a Keras function in this notebook, you will need to write: `K.function(...)`.

## 1 - Problem Statement

Build a car detection system. 
Collect data by mounting a camera to the hood/front of the car.

<center>
<video width="400" height="200" src="../data/nb_images/road_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>  
  
Pictures taken from a car-mounted camera while driving around Silicon Valley.  

We would like to especially thank [drive.ai](https://www.drive.ai/) for providing this dataset! 
Drive.ai is a company building the brains of self-driving vehicles.


<img src="../data/nb_images/driveai.png" width="100px">

Example of what a bounding boxes look like:

<img src="../data/nb_images/box_label.png" width="700px">
<caption><center> <u> **Figure 1** </u>: **Definition of a box**<br> </center></caption>

**Class representation**: 

If you have 80 classes that you want YOLO to recognize, you can represent the class label $c$ either as an integer from 1 to 80, or as an 80-dimensional vector (with 80 numbers) one component of which is 1 and the rest of which are 0. In this notebook, we will use both representations, depending on which is more convenient for a particular step.  

**The YOLO model is very computationally expensive to train, we will load pre-trained weights for you to use.**

## 2 - YOLO

This algorithm "only looks once" at the image in the sense that it requires only one forward propagation pass through the network to make predictions. After non-max suppression, it then outputs recognized objects together with the bounding boxes.

**Boundary boxes:**  
Each of these grid cells is responsible for predicting 5 bounding boxes. A bounding box describes the rectangle that encloses an object.
YOLO also outputs a confidence score that tells us how certain it is that the predicted bounding box actually encloses some object. This score doesn’t say anything about what kind of object is in the box, just if the shape of the box is any good.

For each grid cell, it predicts:
 - B boundary boxes and each box has one box confidence score
 - one object only regardless of the number of boxes B
 - C conditional class probabilities (one per class for the likeliness of the object class)

**Anchor boxes:**  
There are two ways of predicting the bounding boxes:
 - directly predicting the bounding box of the object
  - using a set of pre-defined bounding boxes (Anchor box) to predict the actual bounding box of the object.

YOLOv1 predicts the coordinates of bounding boxes directly using fully connected layers on top of the convolutional feature extractor. But, it makes a significant amount of localisation error. It is easier to predict the offset based on anchor boxes than to predict the co-ordinates directly.


**Dimension clusters:**  
Instead of using predefined anchor boxes, they looked at the bounding boxes in training data (VOC 2007[11], COCO[12]) and performed K-means clustering on those boxes. This resulted in Dimension clusters.

<img src="../data/nb_images/anchor_box.png" width="700px">

### 2.1 - Model details

First things to know:
- The **input** is a batch of images of shape (m, 608, 608, 3)
- The **output** is a list of bounding boxes along with the recognized classes. Each bounding box is represented by 6 numbers $(p_c, b_x, b_y, b_h, b_w, c)$ as explained above. If you expand $c$ into an 80-dimensional vector, each bounding box is then represented by 85 numbers. 

We will use 5 anchor boxes. So you can think of the YOLO architecture as the following:   

**IMAGE (m, 608, 608, 3) -> DEEP CNN -> ENCODING (m, 19, 19, 5, 85)**

Lets look in greater detail at what this encoding represents:
 - Each boundary box contains 5 elements: (x, y, w, h) + a box confidence score. 
  - The confidence score reflects how likely the box contains an object (objectness) and how accurate is the boundary box.
  - We normalize the bounding box width(w) and height(h) by the image width and height. 
  - x and y are offsets to the corresponding cell. Hence, x, y, w and h are all between 0 and 1. 
 - Each cell has 20 conditional class probabilities.
  - The conditional class probability is the probability that the detected object belongs to a particular class (one probability per category for each cell). 

YOLO’s prediction has a shape of (S, S, B × (5 + C)) = (19, 19, 5 x (5 + 80)) = (19, 19, (5 x 85)) = (19, 19, 425).

<img src="../data/nb_images/architecture.png" width="1000px">
<caption><center> <u> **Figure 2** </u>: **Encoding architecture for YOLO**<br> </center></caption><br>

If the center/midpoint of an object falls into a grid cell, that grid cell is responsible for detecting that object.

Since we are using 5 anchor boxes, each of the 19 x 19 cells thus encodes information about 5 boxes. **Anchor boxes are defined only by their width and height.**

For simplicity, we will flatten the last two last dimensions of the shape (19, 19, 5, 85) encoding. So the output of the Deep CNN is (19, 19, 425).

<img src="../data/nb_images/flatten.png" width="1000px">
<caption><center> <u> **Figure 3** </u>: **Flattening the last two last dimensions**<br> </center></caption>

Now, for each box (of each cell) we will compute the following elementwise product and extract a probability that the box contains a certain class.

<img src="../data/nb_images/probability_extraction.png" width="700px">
<caption><center> <u> **Figure 4** </u>: **Find the class detected by each box**<br> </center></caption></br>

We may mix up those scoring and probability terms easily. Here are the mathematical definitions:  

<img src="../data/nb_images/class_prob.png" width="700px">

Here's one way to visualize what YOLO is predicting on an image:
- For each of the 19x19 grid cells, find the maximum of the probability scores (taking a max across both the 5 anchor boxes and across different classes). 
- Color that grid cell according to what object that grid cell considers the most likely.

Doing this results in this picture: 

<img src="../data/nb_images/proba_map.png" width="700px"></br>
<caption><center> <u> **Figure 5** </u>: Each of the 19x19 grid cells colored according to which class has the largest predicted probability in that cell.<br> </center></caption>

Note that this visualization isn't a core part of the YOLO algorithm itself for making predictions; it's just a nice way of visualizing an intermediate result of the algorithm. 



Another way to visualize YOLO's output is to plot the bounding boxes that it outputs. Doing that results in a visualization like this:  

<img src="../data/nb_images/anchor_map.png" width="450px"></br>

<caption><center> <u> **Figure 6** </u></center></caption><br>
    
Each cell gives you 5 boxes. In total, the model predicts: 19x19x5 = 1805 boxes just by looking once at the image (one forward pass through the network)! Different colors denote different classes. 

In the figure above, we plotted only boxes that the model had assigned a high probability to, but this is still too many boxes. You'd like to filter the algorithm's output down to a much smaller number of detected objects. To do so, you'll use non-max suppression. Specifically, you'll carry out these steps: 
- Get rid of boxes with a low score (meaning, the box is not very confident about detecting a class)
- Select only one box when several boxes overlap with each other and detect the same object.



### 2.2 - Filtering with a threshold on class scores

You are going to apply a first filter by thresholding. You would like to get rid of any box for which the class "score" is less than a chosen threshold. 

The model gives you a total of 19x19x5x85 numbers, with each box described by 85 numbers. It'll be convenient to rearrange the (19,19,5,85) (or (19,19,425)) dimensional tensor into the following variables:  
- `box_confidence`: tensor of shape $(19 \times 19, 5, 1)$ containing $p_c$ (confidence probability that there's some object) for each of the 5 boxes predicted in each of the 19x19 cells.
- `boxes`: tensor of shape $(19 \times 19, 5, 4)$ containing $(b_x, b_y, b_h, b_w)$ for each of the 5 boxes per cell.
- `box_class_probs`: tensor of shape $(19 \times 19, 5, 80)$ containing the detection probabilities $(c_1, c_2, ... c_{80})$ for each of the 80 classes for each of the 5 boxes per cell.

**Exercise**: Implement `yolo_filter_boxes()`.
1. Compute box scores by doing the elementwise product as described in Figure 4. The following code may help you choose the right operator: 
```python
a = np.random.randn(19*19, 5, 1)
b = np.random.randn(19*19, 5, 80)
c = a * b # shape of c will be (19*19, 5, 80)
```
2. For each box, find:
    - the index of the class with the maximum box score ([Hint](https://keras.io/backend/#argmax)) (Be careful with what axis you choose; consider using axis=-1)
    - the corresponding box score ([Hint](https://keras.io/backend/#max)) (Be careful with what axis you choose; consider using axis=-1)
3. Create a mask by using a threshold. As a reminder: `([0.9, 0.3, 0.4, 0.5, 0.1] < 0.4)` returns: `[False, True, False, False, True]`. The mask should be True for the boxes you want to keep. 
4. Use TensorFlow to apply the mask to box_class_scores, boxes and box_classes to filter out the boxes we don't want. You should be left with just the subset of boxes you want to keep. ([Hint](https://www.tensorflow.org/api_docs/python/tf/boolean_mask))

Reminder: to call a Keras function, you should use `K.function(...)`.

In [ ]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .5):
    """Filters YOLO boxes by thresholding on object and class confidence.
    
    Arguments:
    box_confidence -- tensor of shape (19, 19, 5, 1)
    boxes -- tensor of shape (19, 19, 5, 4)
    box_class_probs -- tensor of shape (19, 19, 5, 80)
    threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    
    Returns:
    scores -- tensor of shape (None,), containing the class probability score for selected boxes
    boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of selected boxes
    classes -- tensor of shape (None,), containing the index of the class detected by the selected boxes
    
    Note: "None" is here because you don't know the exact number of selected boxes, as it depends on the threshold. 
    For example, the actual output size of scores would be (10,) if there are 10 boxes.
    """
    
    # Compute box scores
    box_scores = box_confidence * box_class_probs
    
    # Find the box_classes thanks to the max box_scores, keep track of the corresponding score
    box_classes = K.argmax(box_scores, axis=-1) # max index == class index
    box_class_scores = K.max(box_scores, axis=-1)
    
    # Create a filtering mask based on "box_class_scores" by using "threshold". The mask should have the
    # same dimension as box_class_scores, and be True for the boxes you want to keep (with probability >= threshold)
    filtering_mask = box_class_scores >= threshold
    
    # Apply the mask to scores, boxes and classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_a:
    box_confidence = tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed=1)
    boxes = tf.random_normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
    box_class_probs = tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed=1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold=0.5)
    
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

**Expected Output**:

<table>
    <tr>
        <td>
            **scores[2]**
        </td>
        <td>
           10.7506
        </td>
    </tr>
    <tr>
        <td>
            **boxes[2]**
        </td>
        <td>
           [ 8.42653275  3.27136683 -0.5313437  -4.94137383]
        </td>
    </tr>

    <tr>
        <td>
            **classes[2]**
        </td>
        <td>
           7
        </td>
    </tr>
        <tr>
        <td>
            **scores.shape**
        </td>
        <td>
           (?,)
        </td>
    </tr>
    <tr>
        <td>
            **boxes.shape**
        </td>
        <td>
           (?, 4)
        </td>
    </tr>

    <tr>
        <td>
            **classes.shape**
        </td>
        <td>
           (?,)
        </td>
    </tr>

</table>

### 2.3 - Non-max suppression ###

Even after filtering by thresholding over the classes scores, you still end up a lot of overlapping boxes. A second filter for selecting the right boxes is called non-maximum suppression (NMS). 

<img src="../data/nb_images/non-max-suppression.png" style="width:500px;height:400;">
<caption><center> <u> **Figure 7** </u>: In this example, the model has predicted 3 cars, but it's actually 3 predictions of the same car. Running non-max suppression (NMS) will select only the most accurate (highest probabiliy) one of the 3 boxes. <br> </center></caption>


Non-max suppression uses the very important function called **"Intersection over Union"**, or IoU.  
<img src="../data/nb_images/iou.png" style="width:500px;height:400;">
<caption><center> <u> **Figure 8** </u>: Definition of "Intersection over Union". <br> </center></caption>

**Exercise**: Implement iou(). Some hints:
- In this exercise only, we define a box using its two corners (upper left and lower right): (x1, y1, x2, y2) rather than the midpoint and height/width.
- To calculate the area of a rectangle you need to multiply its height (y2 - y1) by its width (x2 - x1)
- You'll also need to find the coordinates (xi1, yi1, xi2, yi2) of the intersection of two boxes. Remember that:
    - xi1 = maximum of the x1 coordinates of the two boxes
    - yi1 = maximum of the y1 coordinates of the two boxes
    - xi2 = minimum of the x2 coordinates of the two boxes
    - yi2 = minimum of the y2 coordinates of the two boxes
    
In this code, we use the convention that (0,0) is the top-left corner of an image, (1,0) is the upper-right corner, and (1,1) the lower-right corner. 

In [ ]:
def iou(box1, box2):
    """Implement the intersection over union (IoU) between box1 and box2
    
    Arguments:
    box1 -- first box, list object with coordinates (x1, y1, x2, y2)
    box2 -- second box, list object with coordinates (x1, y1, x2, y2)
    """

    # Calculate the (y1, x1, y2, x2) coordinates of the intersection of box1 and box2. 
    # Calculate its Area.
    xi1 = max(box1[0], box2[0])
    yi1 = max(box1[1], box2[1])
    xi2 = min(box1[2], box2[2])
    yi2 = min(box1[3], box2[3])
    inter_area = (yi2 - yi1) * (xi2 - xi1) 

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    box1_area = (box1[3] - box1[1]) * (box1[2] - box1[0])
    box2_area = (box2[3] - box2[1]) * (box2[2] - box2[0])
    union_area = box1_area + box2_area - inter_area
    
    # compute the IoU
    iou = inter_area / union_area

    return iou

In [ ]:
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4) 
print("iou = " + str(iou(box1, box2)))

**Expected Output**:

<table>
    <tr>
        <td>
            **iou = **
        </td>
        <td>
           0.14285714285714285
        </td>
    </tr>

</table>

### Non-max suppression

You are now ready to implement non-max suppression. The key steps are: 
1. Select the box that has the highest score.
2. Compute its overlap with all other boxes, and remove boxes that overlap it more than `iou_threshold`.
3. Go back to step 1 and iterate until there's no more boxes with a lower score than the current selected box.

This will remove all boxes that have a large overlap with the selected boxes. Only the "best" boxes remain.

**Exercise**: Implement yolo_non_max_suppression() using TensorFlow. TensorFlow has two built-in functions that are used to implement non-max suppression (so you don't actually need to use your `iou()` implementation):
- [tf.image.non_max_suppression()](https://www.tensorflow.org/api_docs/python/tf/image/non_max_suppression)
- [K.gather()](https://www.tensorflow.org/api_docs/python/tf/gather)

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes=10, iou_threshold=0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    
    Note: The "None" dimension of the output tensors has obviously to be less than max_boxes. Note also that this
    function will transpose the shapes of scores, boxes, classes. This is made for convenience.
    """
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')           # create tensor for tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor
    
    # Get the list of indices corresponding to boxes you keep
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold)
    
    # Select only nms_indices from scores, boxes and classes
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)
    classes = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

**Expected Output**:

<table>
    <tr>
        <td>
            **scores[2]**
        </td>
        <td>
           6.9384
        </td>
    </tr>
    <tr>
        <td>
            **boxes[2]**
        </td>
        <td>
           [-5.299932    3.13798141  4.45036697  0.95942086]
        </td>
    </tr>

    <tr>
        <td>
            **classes[2]**
        </td>
        <td>
           -2.24527
        </td>
    </tr>
        <tr>
        <td>
            **scores.shape**
        </td>
        <td>
           (10,)
        </td>
    </tr>
    <tr>
        <td>
            **boxes.shape**
        </td>
        <td>
           (10, 4)
        </td>
    </tr>

    <tr>
        <td>
            **classes.shape**
        </td>
        <td>
           (10,)
        </td>
    </tr>

</table>

### 2.4 Wrapping up the filtering

It's time to implement a function taking the output of the deep CNN (the 19x19x5x85 dimensional encoding) and filtering through all the boxes using the functions you've just implemented. 

**Exercise**: Implement `yolo_eval()` which takes the output of the YOLO encoding and filters the boxes using score threshold and NMS. There's just one last implementational detail you have to know. There're a few ways of representing boxes, such as via their corners or via their midpoint and height/width. YOLO converts between a few such formats at different times, using the following functions (which we have provided): 

```python
boxes = yolo_boxes_to_corners(box_xy, box_wh) 
```
which converts the yolo box coordinates (x,y,w,h) to box corners' coordinates (x1, y1, x2, y2) to fit the input of `yolo_filter_boxes`
```python
boxes = scale_boxes(boxes, image_shape)
```
YOLO's network was trained to run on 608x608 images. If you are testing this data on a different size image--for example, the car detection dataset had 720x1280 images--this step rescales the boxes so that they can be plotted on top of the original 720x1280 image.  

Don't worry about these two functions; we'll show you where they need to be called.  

In [ ]:
def scale_boxes(boxes, orig_image_shape):
    """ Scales the predicted boxes back to original image shape in order to be drawable on the image"""
    height, width = orig_image_shape[0], orig_image_shape[1]
    image_dims = K.stack([height, width, height, width])
    image_dims = K.reshape(image_dims, [1, 4])
    boxes = boxes * image_dims
    return boxes

In [ ]:
def yolo_eval(yolo_outputs, image_shape=(720., 1280.), max_boxes=10, score_threshold=.5, iou_threshold=.5):
    """
    Converts the output of YOLO encoding (a lot of boxes) to your predicted boxes along with their scores, box coordinates and classes.
    
    Arguments:
    yolo_outputs -- output of the encoding model (for image_shape of (608, 608, 3)), contains 4 tensors:
                    box_confidence: tensor of shape (None, 19, 19, 5, 1)
                    box_xy: tensor of shape (None, 19, 19, 5, 2)
                    box_wh: tensor of shape (None, 19, 19, 5, 2)
                    box_class_probs: tensor of shape (None, 19, 19, 5, 80)
    image_shape -- tensor of shape (2,) containing the input shape, in this notebook we use (608., 608.) (has to be float32 dtype)
    max_boxes -- integer, maximum number of predicted boxes you'd like
    score_threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (None, ), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
    """
    
    # Retrieve outputs of the YOLO model
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions 
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Use one of the functions you've implemented to perform Score-filtering with a threshold of score_threshold
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)
    
    # Scale boxes back to original image shape.
    boxes = scale_boxes(boxes, image_shape)

    # Use one of the functions you've implemented to perform Non-max suppression with a threshold of iou_threshold
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
    scores, boxes, classes = yolo_eval(yolo_outputs)
    
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

**Expected Output**:

<table>
    <tr>
        <td>
            **scores[2]**
        </td>
        <td>
           138.791
        </td>
    </tr>
    <tr>
        <td>
            **boxes[2]**
        </td>
        <td>
           [ 1292.32971191  -278.52166748  3876.98925781  -835.56494141]
        </td>
    </tr>

    <tr>
        <td>
            **classes[2]**
        </td>
        <td>
           54
        </td>
    </tr>
        <tr>
        <td>
            **scores.shape**
        </td>
        <td>
           (10,)
        </td>
    </tr>
    <tr>
        <td>
            **boxes.shape**
        </td>
        <td>
           (10, 4)
        </td>
    </tr>

    <tr>
        <td>
            **classes.shape**
        </td>
        <td>
           (10,)
        </td>
    </tr>

</table>

<font color='blue'>
**Summary for YOLO**:
- Input image (608, 608, 3)
- The input image goes through a CNN, resulting in a (19,19,5,85) dimensional output. 
- After flattening the last two dimensions, the output is a volume of shape (19, 19, 425):
    - Each cell in a 19x19 grid over the input image gives 425 numbers. 
    - 425 = 5 x 85 because each cell contains predictions for 5 boxes, corresponding to 5 anchor boxes, as seen in lecture. 
    - 85 = 5 + 80 where 5 is because $(p_c, b_x, b_y, b_h, b_w)$ has 5 numbers, and and 80 is the number of classes we'd like to detect
- You then select only few boxes based on:
    - Score-thresholding: throw away boxes that have detected a class with a score less than the threshold
    - Non-max suppression: Compute the Intersection over Union and avoid selecting overlapping boxes
- This gives you YOLO's final output. 

## 3 - Test YOLO pretrained model on images

In this part, you are going to use a pretrained model and test it on the car detection dataset. As usual, you start by **creating a session to start your graph**. Run the following cell.

In [ ]:
sess = K.get_session()

### 3.1 - Defining classes, anchors and image shape.

Recall that we are trying to detect 80 classes, and are using 5 anchor boxes. We have gathered the information about the 80 classes and 5 boxes in two files "coco_classes.txt" and "yolo_anchors.txt". Let's load these quantities into the model by running the next cell. 

The car detection dataset has 720x1280 images, which we've pre-processed into 608x608 images. 

In [ ]:
def read_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [ ]:
def read_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        anchors = np.array(anchors).reshape(-1, 2)
    return anchors

In [ ]:
class_names = read_classes("../data/model_data/coco_classes.txt")
anchors = read_anchors("../data/model_data/yolo_anchors.txt")
image_shape = (720., 1280.)    

class_names[:5]
anchors

In [ ]:
def generate_colors(class_names):
    """Generate random colors for drawing bounding boxes."""
    import colorsys, random
    
    hsv_tuples = [(x / len(class_names), 1., 1.) for x in range(len(class_names))]
    colors = [colorsys.hsv_to_rgb(h, s, v) for (h, s, v) in hsv_tuples]
    colors = [(int(r * 255), int(g * 255), int(b * 255)) for (r, g, b) in colors]
    
    random.seed(10101)      # Fixed seed for consistent colors across runs.
    random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
    random.seed(None)       # Reset seed to default.
    return colors

colors = generate_colors(class_names)
colors[:5]

### 3.2 - Loading a pretrained model

Training a YOLO model takes a very long time and requires a fairly large dataset of labelled bounding boxes for a large range of target classes. You are going to load an existing pretrained Keras YOLO model stored in "yolo.h5". (These weights come from the official YOLO website, and were converted using a function written by Allan Zelener. References are at the end of this notebook. Technically, these are the parameters from the "YOLOv2" model, but we will more simply refer to it as "YOLO" in this notebook.) Run the cell below to load the model from this file.

In [ ]:
# !cd ~
# !cd _lauthom_intel_design_studio/YAD2K-master
# !python yad2k.py yolo.cfg yolo.weights ../data/model_data/yolo.h5
# ./yad2k.py yolov2_608.cfg yolov2_608.weights model_data/yolo_608.h5

In [ ]:
yolo_model = load_model("./model_data/yolo_608.h5")

This loads the weights of a trained YOLO model. Here's a summary of the layers your model contains.

In [ ]:
yolo_model.summary()

**Note**: On some computers, you may see a warning message from Keras. Don't worry about it if you do--it is fine.

**Reminder**: this model converts a preprocessed batch of input images (shape: (m, 608, 608, 3)) into a tensor of shape (m, 19, 19, 5, 85) as explained in Figure (2).

### 3.3 - Convert output of the model to usable bounding box tensors

The output of `yolo_model` is a (m, 19, 19, 5, 85) tensor that needs to pass through non-trivial processing and conversion. The following cell does that for you.

In [ ]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

You added `yolo_outputs` to your graph. This set of 4 tensors is ready to be used as input by your `yolo_eval` function.

### 3.4 - Filtering boxes

`yolo_outputs` gave you all the predicted boxes of `yolo_model` in the correct format. You're now ready to perform filtering and select only the best boxes. Lets now call `yolo_eval`, which you had previously implemented, to do this. 

In [ ]:
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

### 3.5 - Run the graph on an image

Let the fun begin. You have created a (`sess`) graph that can be summarized as follows:

1. <font color='purple'> yolo_model.input </font> is given to `yolo_model`. The model is used to compute the output <font color='purple'> yolo_model.output </font>
2. <font color='purple'> yolo_model.output </font> is processed by `yolo_head`. It gives you <font color='purple'> yolo_outputs </font>
3. <font color='purple'> yolo_outputs </font> goes through a filtering function, `yolo_eval`. It outputs your predictions: <font color='purple'> scores, boxes, classes </font>

**Exercise**: Implement predict() which runs the graph to test YOLO on an image.
You will need to run a TensorFlow session, to have it compute `scores, boxes, classes`.

The code below also uses the following function:
```python
image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
```
which outputs:
- image: a python (PIL) representation of your image used for drawing boxes. You won't need to use it.
- image_data: a numpy-array representing the image. This will be the input to the CNN.

**Important note**: when a model uses BatchNorm (as is the case in YOLO), you will need to pass an additional placeholder in the feed_dict {K.learning_phase(): 0}.

### Predict on file

In [ ]:
def predict(sess, image_file):
    """
    Runs the graph stored in "sess" to predict boxes for "image_file". Prints and plots the preditions.
    
    Arguments:
    sess -- your tensorflow/Keras session containing the YOLO graph
    image_file -- name of an image stored in the "images" folder.
    
    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes
    
    Note: "None" actually represents the number of predicted boxes, it varies between 0 and max_boxes. 
    """
    
    # Preprocess your image
    image, image_data = preprocess_image("images/" + image_file, model_image_size=(416, 416))
    
    # Run the session with the correct tensors and choose the correct placeholders in the feed_dict.
    # You'll need to use feed_dict={yolo_model.input: ... , K.learning_phase(): 0})
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict={
                                                            yolo_model.input: image_data, 
                                                            K.learning_phase(): 0})
    
    # Generate colors for drawing bounding boxes.
#     colors = generate_colors(class_names)
    
    # Draw bounding boxes on the image file if any
    if len(out_classes) > 0:
        img = draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
        image = Image.fromarray(img, 'RGB')
    
    # Print predictions info
    plt.gca().set_title('Found {} boxes for {}'.format(len(out_boxes), image_file))
    
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=90)
    
    # Image to display in the notebook
    output_image = imread(os.path.join("out", image_file))
    
    return out_scores, out_boxes, out_classes, output_image

#### Draw bounding boxes

In [ ]:
def draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors):
    """Draw bounding boxes if any class predictions available.
    """
    
    try:
        rev_classes = reversed(list(enumerate(out_classes)))
    except:
        return None

    img = np.array(image)
    
    for i, c in rev_classes:
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]

        # Keep border within imgage
        top, left, bottom, right = ((box + 0.5)//1).astype('int32')
        top = max(0, top)
        left = max(0, left)
        bottom = min(image.size[1], bottom)
        right = min(image.size[0], right)
        
        # Draw bounding box
        img = cv2.rectangle(img, (left, top), (right, bottom), colors[c], 3)
        
        # Draw text box
        label = '{} {:.2f}'.format(predicted_class, score)
        label_width = cv2.getTextSize(label, 1, 1.7, 1)[0][0]
        img = cv2.rectangle(img, (left - 2, top - 35), (left + label_width+5, top+1), colors[c], -3)
        img = cv2.putText(img, label, (left + 2, top - 10), 1, 1.7, (255, 255, 255), 2, cv2.LINE_AA)
    
    return img

### Predict sequence of images

Run the following cell on the "test.jpg" image to verify that your function is correct.

In [ ]:
out_scores, out_boxes, out_classes, out_image = predict(sess, "test.jpg")
_ = imshow(out_image)
plt.show();

**Expected Output**:

<table>
    <tr>
        <td>
            **Found 7 boxes for test.jpg**
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.60 (925, 285) (1045, 374)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.66 (706, 279) (786, 350)
        </td>
    </tr>
    <tr>
        <td>
            **bus**
        </td>
        <td>
           0.67 (5, 266) (220, 407)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.70 (947, 324) (1280, 705)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.74 (159, 303) (346, 440)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.80 (761, 282) (942, 412)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.89 (367, 300) (745, 648)
        </td>
    </tr>
</table>

In [ ]:
for filename in os.listdir('./images'):
    if filename.endswith(".jpg") and filename.startswith("001"):
        out_scores, out_boxes, out_classes, out_image = predict(sess, filename)
        _ = imshow(out_image)
        plt.show();

## Video prediction

In [ ]:
def predict_seq(frame):
    """
    Runs the graph stored in "sess" to add class prediction(s) and bounding box(es) on image
    
    Global vars:
    sess -- your tensorflow/Keras session containing the YOLO graph
    class_names -- class labels
    colors -- different color per class for bounding boxes 
    
    Arguments:
    image -- image
    
    Returns:
    image with boxes
    
    """
    
    # Preprocess image
    # 1. resize to fit model input size
    # 2. Normalise pixel values
    # 3. Add batch dimension
    resized_image = Image.fromarray(frame, 'RGB').resize(tuple(reversed((416,416))), Image.BICUBIC)
    image_data = np.array(resized_image, dtype='float32') / 255.
    image_data = np.expand_dims(image_data, 0)
    
    # Run the session and feed_dict.
    # K.learning_phase(): 0 => use dropout at training but not at validating
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], feed_dict={
                                                            yolo_model.input: image_data, 
                                                            K.learning_phase(): 0})
    
    # Draw bounding boxes on the image
    if len(out_classes) > 0:
        img = draw_boxes(Image.fromarray(frame, 'RGB'), out_scores, out_boxes, out_classes, class_names, colors)
        frame = Image.fromarray(img, 'RGB')
    
    return img 

### Stream video frames to model

##### NOTE: `clip.fl_image(predict)` expects color images!!

In [ ]:
def frame2bb(frame):
    """Test predict_seq function. """
    
    try:
        # Transform frame
        img = Image.fromarray(frame, 'RGB').resize(tuple((416, 416)), Image.BICUBIC)
        image_data = np.array(img, dtype='float32') / 255.
        image_data = np.expand_dims(image_data, 0)
        
        # Run model in session
        out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes], 
                                                      feed_dict={yolo_model.input: image_data, 
                                                                 K.learning_phase(): 0})
        # Draw bounding boxes in frame
        if len(out_classes) > 0:
            frame = draw_boxes(Image.fromarray(frame, 'RGB'), out_scores, out_boxes, out_classes, class_names, colors)
    except:
        pass
    
    return frame

#### Test frames

In [ ]:
project_video_output = './project_video_output.mp4'
vclip = VideoFileClip("./project_video.mp4") # w*h*3 RGB array

for i, frame in enumerate(vclip.iter_frames()):
    if (i > 300) and (i < 308):
        img = frame2bb(frame)
        _ = imshow(Image.fromarray(img, 'RGB')) 
        plt.show();

#### Create video

Time to create 50sec video is: 15min 15sec

In [ ]:
lane_clip = vclip.fl_image(frame2bb)  # test predict_seq                    
%time lane_clip.write_videofile(project_video_output, audio=False)

**Watch the video you just created:  **

<center>
<video width="720" height="360" src="./project_video_output.mp4" type="video/mp4" controls>
</video>
</center>

<font color='blue'>
**What you should remember**:
- YOLO is a state-of-the-art object detection model that is fast and accurate
- It runs an input image through a CNN which outputs a 19x19x5x85 dimensional volume. 
- The encoding can be seen as a grid where each of the 19x19 cells contains information about 5 boxes.
- You filter through all the boxes using non-max suppression. Specifically: 
    - Score thresholding on the probability of detecting a class to keep only accurate (high probability) boxes
    - Intersection over Union (IoU) thresholding to eliminate overlapping boxes
- Because training a YOLO model from randomly initialized weights is non-trivial and requires a large dataset as well as lot of computation, we used previously trained model parameters in this exercise. If you wish, you can also try fine-tuning the YOLO model with your own dataset, though this would be a fairly non-trivial exercise. 

**References**: The ideas presented in this notebook came primarily from the two YOLO papers. The implementation here also took significant inspiration and used many components from Allan Zelener's github repository. The pretrained weights used in this exercise came from the official YOLO website. 
- Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi - [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640) (2015)
- Joseph Redmon, Ali Farhadi - [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) (2016)
- Allan Zelener - [YAD2K: Yet Another Darknet 2 Keras](https://github.com/allanzelener/YAD2K)
- The official YOLO website (https://pjreddie.com/darknet/yolo/) 

**Car detection dataset**:
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">The Drive.ai Sample Dataset</span> (provided by drive.ai) is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>. We are especially grateful to Brody Huval, Chih Hu and Rahul Patel for collecting and providing this dataset. 